In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Load data from IMDb dataset


2024-01-23 21:44:00.105103: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 21:44:00.145721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 21:44:00.145780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 21:44:00.146943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 21:44:00.153974: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 21:44:00.154504: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [18]:

file_path = '/home/siddhantp/Desktop/New Folder/archive/IMDB Dataset.csv'
df = pd.read_csv(file_path)

# Assume your CSV file has columns 'review' and 'label'
texts = df['review'].tolist()
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
labels= df['sentiment'].tolist()



In [19]:
# Tokenize and vectorize the text
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')



In [20]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Get vocabulary size and embedding dimension
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 16


In [21]:

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=100))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [22]:
# Display the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 16)           1988064   
                                                                 
 lstm_2 (LSTM)               (None, 100, 64)           20736     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2050273 (7.82 MB)
Trainable params: 2050

In [23]:
# Train the model
y_train = np.array(y_train)
y_test = np.array(y_test)
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1250/1250 [==============================] - 70s 54ms/step - loss: 0.4843 - accuracy: 0.7614 - val_loss: 0.3923 - val_accuracy: 0.8284
Epoch 2/5
1250/1250 [==============================] - 68s 55ms/step - loss: 0.3646 - accuracy: 0.8448 - val_loss: 0.3789 - val_accuracy: 0.8358
Epoch 3/5
1250/1250 [==============================] - 67s 54ms/step - loss: 0.3320 - accuracy: 0.8601 - val_loss: 0.3936 - val_accuracy: 0.8316
Epoch 4/5
1250/1250 [==============================] - 67s 54ms/step - loss: 0.3058 - accuracy: 0.8707 - val_loss: 0.3958 - val_accuracy: 0.8243
Epoch 5/5
1250/1250 [==============================] - 67s 54ms/step - loss: 0.2758 - accuracy: 0.8860 - val_loss: 0.4006 - val_accuracy: 0.8271


In [24]:
# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))



313/313 [==============================] - 6s 16ms/step
              precision    recall  f1-score   support

           0       0.81      0.85      0.83      4961
           1       0.84      0.80      0.82      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000

